In [2]:
!pip install twython
!pip install apscheduler

     |████████████████████████████████| 147 kB 18.3 MB/s eta 0:00:01     |█████████████████████████████   | 133 kB 18.3 MB/s eta 0:00:01


In [2]:
import os
import datetime
import json
import twython
import json

In [3]:
import pandas as pd
import numpy as np 

In [4]:
from twython import Twython
from contextlib import suppress
from requests_oauthlib import OAuth1Session
from apscheduler.schedulers.background import BackgroundScheduler as Scheduler

In [4]:
# THIS LINE OF CODE IS WHAT WE USED TO AUTHENTICATE AND MAKE REQUESTS

# Enter your keys/secrets as strings in the following fields
# credentials = {}
# credentials['CONSUMER_KEY'] = "consumer key"
# credentials['CONSUMER_SECRET'] = "consumer token"
# credentials['ACCESS_KEY'] = "access key"
# credentials['ACCESS_SECRET'] = "access token"

# Save the credentials object to file
# with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)

In [5]:
# Load credentials from json file
with open("/Users/tlipman/.secret/twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [6]:
# getting the header for our dataframe so we have something to append to 
startdf = pd.read_csv('df')
# when our dataframe is created the index is unnamed, when you export it is renamed so we need to fix it so our df realizes it's the same 
startdf = startdf.head(0).rename(columns={'Unnamed: 0':''})
#assign our id as the most recent tweet id to start so that we get everything after it 
id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']

def myfn(q, name):
    # import global variables we will be using this function to alter
    global id_
    global startdf 
    # Create our query
    query = {'q': q,
            'result_type': 'recent',
            'count': 100,
            'lang': 'en',
            'max_id': id_,
            'entities': {
                "hashtags": [],
                "symbols": [],
                "user_mentions": []
            }
            }
    #build our query into a dictionary to easily turn into dataframe 
    dict_ = {'user': [], 'user_id':[], 'post_id': [], 'text': [], 'favorite_count': [], 'hashtags':[], 'symbols':[], 'user_mentions':[], 
             'retweet_count':[]}
    for status in python_tweets.search(**query)['statuses']:
        dict_['user'].append(status['user']['screen_name'])
        dict_['user_id'].append(status['user']['id'])
        dict_['text'].append(status['text'])
        dict_['hashtags'].append(status['entities']['hashtags'])
        dict_['symbols'].append(status['entities']['symbols'])
        dict_['user_mentions'].append(status['entities']['user_mentions'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['retweet_count'].append(status['retweet_count'])
        dict_['post_id'].append(status['id'])
    # put data in a DataFrame to work with it easier
    df = pd.DataFrame(dict_)
    #removing our vairables stuck in dictionaries within our dataframe and give them their own columns
    df['mentions'] = ' '
    df['hashtag'] = ' '
    for i in range(len(df)):
        for j in range(len(df.at[i, 'user_mentions'])):
            try:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions']) + ' ' + str(df.at[i, 'user_mentions'][j]['id'])
            except:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions'])
        for k in range(len(df.at[i, 'hashtags'])):
            try:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag']) + ' ' + str(df.at[i, 'hashtags'][k]['text'])
            except:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag'])
    # get rid of columns that we took everything we need from 
    df = df.drop(['user_mentions', 'hashtags'], 1)
    startdf = startdf.append(df)
    # not sure why id_ doesn't want to change unless reset but do it because it works and can't hurt
    id_ = None
    # get oldest id_ (lowest number) and subtract one so that that you don't reinclude it in the next search
    # reset id_ to be oldest
    id_ = int(df['post_id'].sort_values(ascending=True).to_frame().reset_index()['post_id'][0] - 1) 
    return startdf, id_

In [7]:
query_list1 = ['EXO', 'spacehotel', 'hubble', 'spaceprogram', 'thesolarsystem',
              'PerseveranceRover', 'SpaceExploration', 'lifeinspace',
              'voyagerstation', 'exoplanet', 'space']

In [ ]:
query_list0 = ['ExoMars', 'HubbleGotchu', 'JWST', 'Mars', 'Mars2021',
              'PerseveranceRover', 'SpaceExploration', 'astrophotography',
              'perseverance', 'exoplanet', 'space']

In [6]:
query_list = ['SpaceExploration', 'perseverance', 'exoplanet', 'space', 'EXO', 'spacehotel',
               'hubble', 'spaceprogram', 'thesolarsystem', 'lifeinspace', 'voyagerstation']

In [8]:
# eventual goal is to use those loop to do all of our searches
end_item = datetime.datetime.now()

for item in query_list1:
    #reset id_ to be most recent tweet for each item
    id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})
    id_ = id_['statuses'][0]['id']
    # for each item, reset the start time to the end of the last item 
    start_time = end_item + datetime.timedelta(seconds=1)
    # set the end time to be 90 min after you start, gathering 100 tweets every 100 seconds for 2,700 tweets from each item in list
    end_item = end_item + datetime.timedelta(seconds=5401)
    #print start and end times so I know when the program will be finished and where it should be at 
    print(f'starting {item} time : {start_time.strftime("%H:%M:%S")}, ending time: {end_item.strftime("%H:%M:%S")}')
    #occassionally getting key/value errors of 0 that I cannot find the cause of, but do not affect anything in how our dataframe
    #is made, I just don't want them printing 
    with suppress(KeyError, ValueError):
        #initiate scheduler
        sch = Scheduler()
        #add our function and items, we're doing 50/10 seconds 
        sch.add_job(myfn, 'interval', (item, startdf), seconds=10, start_date=start_time, end_date=end_item)
        #start scheduler
        sch.start()

starting EXO time : 09:22:16, ending time: 10:52:16
starting spacehotel time : 10:52:17, ending time: 12:22:17
starting hubble time : 12:22:18, ending time: 13:52:18
starting spaceprogram time : 13:52:19, ending time: 15:22:19
starting thesolarsystem time : 15:22:20, ending time: 16:52:20
starting PerseveranceRover time : 16:52:21, ending time: 18:22:21
starting SpaceExploration time : 18:22:22, ending time: 19:52:22
starting lifeinspace time : 19:52:23, ending time: 21:22:23
starting voyagerstation time : 21:22:24, ending time: 22:52:24
starting exoplanet time : 22:52:25, ending time: 00:22:25
starting space time : 00:22:26, ending time: 01:52:26


/Users/tlipman/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [ ]:
# Used to check how many lines of code the dataframe is creating
len(startdf)

In [15]:
# Used to view the most popular hashtags
startdf['hashtag'].value_counts().nlargest(50)

                                                           246266
  SpacehotelsTwitterDisco                                    8283
  WomenInScience SpaceExploration                            6676
  PiDay                                                      3557
  OnThisDay                                                  3009
  백현 BAEKHYUN EXO 엑소                                         2958
  OTD Gagarin                                                2574
  Gagarin Vostok1                                            2571
  DYK MarsEra USSR Mars6                                     2380
  EXO Baekhyun NCT Doyoung                                   2183
  NASA SpaceTravel IMAGINEERS ManInSpace                     2168
  EXO                                                        1914
  spacehotel                                                 1803
  InternationalWomensDay                                     1763
  SpaceFactFriday                                            1632
  MessierM

In [26]:
startdf['text'][:25]

0     RT @MichaelGalanin: A collection of spectacula...
1     RT @MichaelGalanin: A collection of spectacula...
2     RT @MichaelGalanin: A collection of spectacula...
3       ExoMars goes for a spin https://t.co/lsZkrvNmo6
4     RT @MichaelGalanin: A collection of spectacula...
5     RT @MichaelGalanin: A collection of spectacula...
6     RT @MichaelGalanin: A collection of spectacula...
7     RT @MichaelGalanin: A collection of spectacula...
8     RT @MichaelGalanin: A collection of spectacula...
9     RT @MichaelGalanin: A collection of spectacula...
10    New session: Perseverance ⟷ ExoMars TGO\nExpec...
11    RT @MichaelGalanin: A collection of spectacula...
12    RT @MichaelGalanin: A collection of spectacula...
13    ExoMars TGO downlinking 50.85 MB from Curiosit...
14    RT @MichaelGalanin: A collection of spectacula...
15    RT @MichaelGalanin: A collection of spectacula...
16    RT @MichaelGalanin: A collection of spectacula...
17    RT @MichaelGalanin: A collection of specta

/Users/tlipman/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
Job "myfn (trigger: interval[0:00:10], next run at: 2021-03-07 18:12:47 EST)" raised an exception
Traceback (most recent call last):
  File "/Users/tlipman/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 4730, in get_value
    return self._engine.get_value(s, k, tz=getattr(series.dtype, "tz", None))
  File "pandas/_libs/index.pyx", line 80, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 88, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hasht

In [1]:
startdf.head()

NameError: name 'startdf' is not defined

In [16]:
# since every time we loop through it starts a new index, there are now multiple entries for each index, we need to fix this
# and the easiest way is to just reset the index once it finishes running 
startdf = startdf.drop_duplicates(subset='post_id')
startdf = startdf.reset_index()

In [54]:
# don't need these columns anymore
startdf = startdf.drop(['index', '', 'symbols'], 1)

In [55]:
# in order to add our strings for our hashtags and mentions together we had to make the entire column a string, sinlucding empty cells
# here we go through and replace empty cells with nan values so pandas will read them as being empty instead of a string
for i in range(len(startdf)):
    if startdf.at[i, 'mentions'] == ' ':
        startdf.at[i, 'mentions'] = np.nan
    else: 
        pass 
    if startdf.at[i, 'hashtag'] == ' ':
        startdf.at[i, 'hashtag'] = np.nan
    else: 
        pass 

In [17]:
startdf.to_csv(r'C:\Users\tlipman\Documents\Flatiron_School\Notes\Phase4\tweets_secondary.csv')